In [1]:
import os, sys, django, pandas as pd , numpy as np
from routes import csv, TJ_FILES_PATH, setup_env
setup_env()

def read_csv(file, compression='infer'):
    return pd.read_csv(TJ_FILES_PATH+file, encoding='latin1',sep=';', compression=compression)

def read_csv_chunck(file, compression='infer'):
    return pd.read_csv(TJ_FILES_PATH+file, encoding='latin1',sep=';', compression=compression,chunksize=5000000)

<code>for chnk in pd.read_csv(TJ_FILES_PATH+csv['processo_unico'], compression='gzip' ,usecols=['ID_PROC','COD_CNJ','COD_PROC'],sep=';',encoding='latin1', low_memory=False, chunksize=5000000):
    print(chnk)<code>

In [11]:
competencia = read_csv(csv['competencia'])

In [13]:
comarca = read_csv('/Parameters/DCP_Comarca.csv')

In [10]:
serventia = read_csv()

{'advogados': '/Advogado/dcpadvogado_2013-2017.csv.gz',
 'tipo_decisao_recurso': '/Parametros/DCP_TIPO_DECISAO_RECURSO.csv.gz',
 'indice_processo': 'PROCCORP_INDICEPROCESSO_2013-2017.csv.gz',
 'assunto': '/Parametros/DCP_ASSUNTO.csv.gz',
 'tipo_decisao': '/Parametros/DCP_TIPO_DECISAO.csv.gz',
 'ato_juiz': '/Parametros/DCP_ATOJUIZ.csv.gz',
 'cardo': '/Parametros/DCP_CARGO.csv.gz',
 'tipo_entidade': '/Parametros/DCP_TIPO_ENTIDADE.csv.gz',
 'classe': '/Parametros/DCP_CLASSE.csv.gz',
 'tipo_andamento': '/Parametros/DCP_TipoAndamento.csv.gz',
 'classe_assunto': '/Parametros/DCP_CLASSEASSUNTO.csv.gz',
 'classe_assunto_1nivel': '/Parametros/DCP_CLASSEASSUNTOPRIMEIRONIVEL.csv.gz',
 'tipo_documento': '/Parametros/PROCOORP_TIPODOCUMENTO.csv.gz',
 'comarca': '/Parametros/DCP_Comarca.csv.gz',
 'rito': '/Parametros/DCP_RITO.csv.gz',
 'serventia': '/Parametros/DCP_Serventias.csv.gz',
 'tipo_andamento_local': '/Parametros/DCP_TIPOANDAMENTOLOCAL.csv.gz',
 'tipo_resultado_audiencia': '/Parametros/dcp_T

In [15]:
comarca_old = read_csv(csv['comarca'])

In [16]:
comarca_old

,COD_COMA,DESC_COMA,COD_TJ,DESC_REDU,NUM_DIAS_ATRASO_DO,IND_DIST_OFIC,IND_VARA_FED,COD_COM_PRINC
0,201.0,Comarca da Capital,1.0,CAP,NaN,N,S,NaN
1,203.0,Regional de Jacarepaguá,203.0,JPA,NaN,S,S,201.0
2,204.0,Regional de Bangu,204.0,BAN,NaN,S,S,201.0
3,208.0,Regional do Méier,208.0,MEI,NaN,S,S,201.0
4,209.0,Regional da Barra da Tijuca,209.0,BTJ,NaN,S,S,201.0
5,220.0,Comarca de Campos dos Goytacazes,14.0,CAM,3.0,N,S,NaN
6,222.0,Comarca de Niterói,2.0,NIT,NaN,N,S,NaN
7,224.0,Comarca de Petrópolis,42.0,PET,1.0,S,S,NaN
8,225.0,Comarca de São Gonçalo,4.0,SGO,NaN,S,S,NaN
9,226.0,Comarca de São João de Meriti,54.0,SJM,NaN,S,S,NaN
